In [2]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
import re
from rich import print
from rich.progress import track
import seaborn as sns
from sklearn.linear_model import Lasso, Ridge, ElasticNet,LassoCV,RidgeCV
from sklearn.model_selection import cross_val_score, train_test_split,KFold, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from selenium.webdriver.common.keys import Keys

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def get_url(search_term):
    """Generate url with search term"""
    
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    search_term = search_term.replace(" ", "+")
    
    
    # Add search term to url
    url = template.format(search_term)
    
    # Add page query to the placeholder
    url += '&page={}'
    
    return url

def extract_record(item):
    """Extract and return data from single record"""
        
    # name and url
    atag = item.h2.a
    name = atag.text.strip()
    new_url = "http://www.amazon.com"+atag.get('href')
    
    
    
    res = [name, new_url]
    return res

def main(search_term, file_name): # File name should have .csv ending
    
    chromedriver = "C:\Program Files\Google\Chrome\Application\chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    DRIVER = webdriver.Chrome(chromedriver)
    

    
    DATA = []
    URL = get_url(search_term)
    
    for page in range(1,10):
        DRIVER.get(URL.format(page))
        SOUP = BeautifulSoup(DRIVER.page_source, 'html.parser')
        RESULTS = SOUP.find_all('div', {'data-component-type': 's-search-result'})
        for item in RESULTS:
            RES = extract_record(item)
            if RES:
                DATA.append(RES)
    
    DRIVER.close()
    
    df = pd.DataFrame(DATA, columns = ['Name', 'URL'])
    df.to_csv(file_name, index=False)

In [ ]:
main("laptop", "Amazon_Laptop.csv")

In [ ]:
df=pd.read_csv('Amazon_Laptop.csv')

In [ ]:
df=df['URL']

In [ ]:
DRIVER = webdriver.Chrome(chromedriver)

In [ ]:
def get_data(url_list:list)->pd.DataFrame:
    lst = list()
    
    for url in track(url_list, description='[green]Stealing in progress...'):
        DRIVER.get(url)
        SOUP = BeautifulSoup(DRIVER.page_source, 'html.parser')
        try:
            #Price
            if SOUP.find('table', id='productDetails_techSpec_section_2').find_all('tr')!= None:
                if SOUP.find('span', id='priceblock_ourprice').text.strip().replace(',', '').replace('$', '')!= None:
                    Price = float(SOUP.find('span', id='priceblock_ourprice').text.strip().replace(',', '').replace('$', ''))
                else:
                    pass
                
                #SUMMARY
                summary_table = SOUP.find('table', id='productDetails_techSpec_section_1').find_all('tr')
                summary_table_data = {x.find('th').text.strip().split('\u200e')[len(x.find('th').text.strip().split('\u200e'))-1]: x.find('td').text.strip().split('\u200e')[len(x.find('td').text.strip().split('\u200e'))-1] for x in summary_table}

                # OTHER DETAILS
                other_technical_details_table = SOUP.find('table', id='productDetails_techSpec_section_2').find_all('tr')
                other_technical_details_table_data = {x.find('th').text.strip().split('\u200e')[len(x.find('th').text.strip().split('\u200e'))-1]: x.find('td').text.strip().split('\u200e')[len(x.find('td').text.strip().split('\u200e'))-1] for x in other_technical_details_table}

                # Merge the two tables in one dictionary then append it to a list
                summary_table_data["Price"]=Price
                lst.append({**summary_table_data,**other_technical_details_table_data})
                
        except:
             print(url)
            
        
        
    return pd.DataFrame(lst)

In [ ]:
df = get_data(df1)

In [ ]:
df= df.rename(columns={'Standing screen display size': 'display_size','Max Screen Resolution':'Resolution','Hard Drive':'Hard_Drive','Average Battery Life (in hours)':'Battery_Life','Item Weight':'Weight','Processor Brand':'Processor_Brand','Processor Count':'Processor_Count','Flash Memory Size':'Hard_Drive_Size','Hard Drive Interface':'Hard_Drive_Type'})

In [ ]:
df['RAM'] = df['RAM'].replace([2,6,1,128,512,256,9],8)

In [ ]:
df['Processor'] = df['Processor'].replace(['1_2GHz_Cortex_A8'],1.2)

In [ ]:
df=df.drop(columns=['Item Dimensions  LxWxH'])

In [ ]:
df=df.drop(columns=['Unnamed: 0', 'Card Description','Rear Webcam Resolution','Graphics Card Ram Size','Voltage','Package Dimensions','Manufacturer','ASIN','Date First Available','Number of USB 2.0 Ports','Computer Memory Type','Screen Resolution','Hardware Platform','Hard Drive Rotational Speed','National Stock Number','Power Source','Optical Drive Type','Product Dimensions','Operating System','Number of USB 3.0 Ports','Item model number','Series','Batteries','Wireless Type','Graphics Coprocessor','Memory Speed'])

In [ ]:
df['Price']=df['Price'].apply(pd.to_numeric)

In [ ]:
df[['display_size','inches']]=df.display_size.str.split(" ",expand=True)

In [ ]:
df['display_size']=df['display_size'].apply(pd.to_numeric)

In [ ]:
df = df.rename(columns={'Processor_Count':'Processor_Count_Cores'})

In [ ]:
df=df.drop(columns=['inches'])

In [ ]:
df['Weight']=df.Weight.str.split(" ",expand=True)

In [ ]:
df['Weight']=df['Weight'].apply(pd.to_numeric)

In [ ]:
df['Processor']=df.Processor.str.split(" ",expand=True)

In [ ]:
df['display_size'].fillna(value=14.0, inplace=True)

In [ ]:
df['RAM']=df.RAM.str.split(" ",expand=True)

In [ ]:
df['RAM']=df['RAM'].apply(pd.to_numeric)

In [ ]:
df['RAM'].fillna(value=8, inplace=True)

In [ ]:
mean_value=df['RAM'].mean()
print('Mean of values in column RAM:')
print(mean_value)

In [ ]:
df['Battery_Life']=df.Battery_Life.str.split(" ",expand=True)

In [ ]:
df['Battery_Life'] = df['Battery_Life'].replace(['11.0,'],11.0)

In [ ]:
df['Battery_Life']=df['Battery_Life'].apply(pd.to_numeric)

In [ ]:
df.Battery_Life.fillna(value=8,inplace=True)

In [ ]:
df['Hard_Drive_']=df.Hard_Drive.str.split(" ",expand=True)

In [ ]:
df['Hard_Drive'] = df['Hard_Drive'].replace(['1','1.5','6','No','1000'],1000)

In [ ]:
df['Hard_Drive'] = df['Hard_Drive'].replace(['Embedded','628','100','PCIe','1024','8','4'],128)

In [ ]:
df['Hard_Drive'] = df['Hard_Drive'].replace(['eMMC'],64)

In [ ]:
df['Hard_Drive'] = df['Hard_Drive'].replace(['192','embedded','2556','hdd','750','SSD'],256)

In [ ]:
df['Hard_Drive']=df['Hard_Drive'].apply(pd.to_numeric)

In [ ]:
df.Hard_Drive.fillna(value=512,inplace=True)

In [ ]:
df.Processor.fillna(value=2.5,inplace=True)

In [ ]:
df=df.drop([1310,1736,1774])

In [ ]:
df=df.drop(columns=['Hard_Drive_Size','Hard_Drive_Type','Chipset_Brand'])

In [ ]:
df.Processor_Count_Cores.fillna(value=4,inplace=True)

In [ ]:
# Intel 
df['Processor_Brand'] = df['Processor_Brand'].replace(['Unknown','Qualcomm','Intel Core i5-1145G7','10th Gen Intel Comet Lake Core i7-1065G7','Intel Core i7-1165G7','Intel Celeron Processor J4115','Intel Core i7-1185G7','NVIDIA','Allwinner','VIA','MedieTek','Mediatek'],'Intel')
# NVIDIA
df['Processor_Brand'] = df['Processor_Brand'].replace([''],'NVIDIA')
# AMD 
df['Processor_Brand'] = df['Processor_Brand'].replace(['AMD Ryzen 5 4600U','AMD Ryzen 5 3500C'],'AMD')

In [ ]:
df.loc[df['Brand']=="N\'"]

In [ ]:
df=df.drop(1094)

In [ ]:
df.Processor_Brand.fillna(value='AMD',inplace=True)

In [ ]:
# Count Resolution
pd.DataFrame(df['Resolution'].value_counts(ascending=True))

In [ ]:
df.Weight.fillna(value=4,inplace=True)

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1920x1080 Pixels','1920 x 1080','1920x1080','1920 x 1080 MP','19201080 Pixels','1920X1080 Pixels','1920 X 1080 Pixels','1920 x 1080 Lines Per Inch','1920x1080 Lines Per Inch','19201080 Lines Per Inch','FHD (1920 x 1080) IPS 144Hz Pixels','1920x1080 MP','Full HD (1920 x 1080) IPS 144Hz Pixels','Full HD (1920 x 1080) 144Hz IPS','1920×1080 Pixels','1920 * 1080 Pixels','1,920x1,080 Pixels','FHD (1920 x 1080) IPS Pixels','FHD (1920 x 1080) WVA 120Hz Pixels','1921 x 1080 Pixels','FHD (1920x1080) IPS 144Hz Pixels','Full HD(1920 x 1080) Anti-glare Pixels','Full HD (1920 x 1080)','1920 x 1080 (Full HD)','1920x1080 Pixels','1920 x 1080','1920x1080','1920 x 1080 MP','19201080 Pixels','1920X1080 Pixels','1920 X 1080 Pixels','1920 x 1080 Lines Per Inch','1920x1080 Lines Per Inch','19201080 Lines Per Inch','FHD (1920 x 1080) IPS 144Hz Pixels','1920x1080 MP','Full HD (1920 x 1080) IPS 144Hz Pixels','Full HD (1920 x 1080) 144Hz IPS','1920×1080 Pixels','1920 * 1080 Pixels','1,920x1,080 Pixels','FHD (1920 x 1080) IPS Pixels','FHD (1920 x 1080) WVA 120Hz Pixels','1921 x 1080 Pixels','FHD (1920x1080) IPS 144Hz Pixels','Full HD(1920 x 1080) Anti-glare Pixels','Full HD (1920 x 1080)','1920 x 1080 (Full HD)','1080p Full HD Pixels','1080 Pixels','1290 x 1080','1920*1080 Lines Per Inch','FHD Display','1920*1080 Pixels'],'1920 x 1080 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1366x768 Pixels','1366x768','1366X768 Pixels','1366 x 768 MP','HD Display','1366 x 768 Lines Per Inch','1366*768 Pixels','HD (1366 x 768) Touchscreen Pixels','1366X768 pixels','1366X768 pixels','1368x766 Pixels','1366 X 768','1366x768 MP','1361 x 768 Pixels','1366x766 Pixels','1366 X 768 MP','1366 x 768 HD Pixels','HD (1366 x 768) SVA Pixels','1366×768 Pixels','HD ( 1366 x 768 )','HD (1366 x 768) Pixels','1366 Pixels','1366*768 pixels','1366x768 pixels','1366 x 768'],'1366x768 pixels'

In [ ]:
df['Resolution'] = df['Resolution'].replace(['2160 X 1440 Pixels','2560 x 1440 Pixels','2560 x 1600 Pixels','2560 x 1440','2560x1440 Pixels','2160 x 1350 Pixels','2048 x 1536','2048 x 1536','2256 x 1504','2160x1440 Pixels','2160*1440 Pixels','2560 x 1600 Lines Per Inch','2666 Pixels','2496 x 1664','2560 x 1600','2256 x 1504 Pixels','4096 x 2304 Pixels'],'2K')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1600 x 900 Pixels','1,600 x 900 Pixels','HD+ Display','1600 x 900 Lines Per Inch','1600x900 MP','HD+ (1600 x 900) Pixels','1600 x 900 Pixels','1600 x 900','1600x900','1600x900 Pixels','1600 x 900 Pixels'],'1600 x 900 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['3840 x 2160','3840 x 2400 Pixels','3000 x 2000 Pixels','4096×2304 @24Hz Lines Per Inch','3240 x 2160','3200 x 1800 Pixels','4K Lines Per Inch','3200X1800 pixels','3300 x 2200 Pixels','3840 x 2160','3840 x 2160 Pixels','3840 x 2400','4K UHD (3840 x 2160)','3840X2160 Pixels','3840 x 2160','3840 x 2400 Pixels','3000 x 2000 Pixels','4096×2304 @24Hz Lines Per Inch','3240 x 2160'],'4K')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1920 x 1200','1900*1200 Pixels','1920x1200 Pixels','1920X1200 Pixels','1920*1200 Pixels','1920 x 1200 Pixels'],'1920 X 1200 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1600 x 900','1600x900','1600x900 Pixels','1600 x 900 Pixels'],'1600 x 900 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1280×720 Pixels','1280 x 720 Pixels','1280 x 720','1280 X 800 Pixels'],'1280×720 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1024*600 Pixels','1024 Lines Per Inch'],'1024*600')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1366 x 912 Pixels','1366 x 912 MP'],'1366 x 912 Pixels')

In [ ]:
df['Resolution'] = df['Resolution'].replace(['1366 x 912 Pixels','1366 x 768 Pixels'],'1366 x 768 Pixels')

In [ ]:
df.Resolution.fillna(value='1280×720 Pixels', inplace=True, limit=126)

In [ ]:
df.Resolution.fillna(value='4K', inplace=True, limit=126)

In [ ]:
df.Resolution.fillna(value='1920 x 1080 Pixels', inplace=True, limit=126)

In [ ]:
df.Resolution.fillna(value='1366 x 912 Pixels', inplace=True, limit=126)

In [ ]:
df.Resolution.fillna(value='2K', inplace=True, limit=126)

In [ ]:
df['Color'] = df['Color'].replace(['Mica','Almond'],'Others')

In [ ]:
# Count Resolution
pd.DataFrame(df['Resolution'].value_counts(ascending=True))

In [ ]:
# Color
m = df['Color'].str.contains('Gray', na=False)
df.loc[m,'Color'] = 'Gray'
m = df['Color'].str.contains('Gery', na=False)
df.loc[m,'Color'] = 'Gray'
m = df['Color'].str.contains('Grey', na=False)
df.loc[m,'Color'] = 'Gray'
m = df['Color'].str.contains('grey', na=False)
df.loc[m,'Color'] = 'Gray'
m = df['Color'].str.contains('Black', na=False)
df.loc[m,'Color'] = 'Black'
m = df['Color'].str.contains('Blue', na=False)
df.loc[m,'Color'] = 'Blue'
m = df['Color'].str.contains('Silver', na=False)
df.loc[m,'Color'] = 'Silver'
m = df['Color'].str.contains('Gold', na=False)
df.loc[m,'Color'] = 'Gold' 
m = df['Color'].str.contains('White', na=False)
df.loc[m,'Color'] = 'White'
m = df['Color'].str.contains('Red', na=False)
df.loc[m,'Color'] = 'Red'
m = df['Color'].str.contains('Sliver', na=False)
df.loc[m,'Color'] = 'Silver'
m = df['Color'].str.contains('gold', na=False)
df.loc[m,'Color'] = 'Gold'
df['Color'] = df['Color'].replace(['GTX 1660 Ti','Windows 10 Pro','4G','Dark blue','Core-i5','Machined Aluminum','Jet black','Electro Punk','灰色','SLIVER','黑色','Intel Celeron N4020','Non-Touchscreen','AORUS 15G - Alloy + Mech Keyboard','Dark Orchid','BLACK, GREEN','Navy','Snowflake white','Citrus Soda Yellow','Non-Touch | i3-7130U | 128GB SSD','Light Titan','Forest teal with matte finish','Carbon Fiber','15 inch FHD','Bronze','15.6" | AERO 15 | TOP SELLER','Y13Pro/13.3inch','Dune','M16-i9-11900H-RTX 3060-16G/1T','Core black','Natural silver','NEW','Flaming Copper','银色','Forest Teal','Sandstorm','Platinum Titan','Purple','Green','Rose Pink','Sand'],'Black')

In [ ]:
df['Color'] = df['Color'].replace(['Mica','Almond'],'Others')

In [ ]:
df.Resolution.fillna(value='Almond', inplace=True, limit=11)

In [ ]:
df.Resolution.fillna(value='Others', inplace=True, limit=250)

In [ ]:
df.Resolution.fillna(value='Silver', inplace=True, limit=100)

In [ ]:
df.Resolution.fillna(value='White', inplace=True, limit=100)

In [ ]:
df.Resolution.fillna(value='Blue', inplace=True, limit=100)

In [ ]:
df.Resolution.fillna(value='Gold', inplace=True, limit=150)

In [ ]:
df=df.drop(817)

In [ ]:
df['Processor_Count_Cores'] = df['Processor_Count_Cores'].replace([12],8.0)

In [ ]:
# Count RAM
pd.DataFrame(df['RAM'].value_counts(ascending=True))

In [ ]:
# Intel 
df['Processor_Brand'] = df['Processor_Brand'].replace(['Unknown','Qualcomm','Intel Core i5-1145G7','10th Gen Intel Comet Lake Core i7-1065G7','Intel Core i7-1165G7','Intel Celeron Processor J4115','Intel Core i7-1185G7','NVIDIA','Allwinner','VIA','MedieTek','Mediatek'],'Intel')
# NVIDIA
df['Processor_Brand'] = df['Processor_Brand'].replace([''],'NVIDIA')
# AMD 
df['Processor_Brand'] = df['Processor_Brand'].replace(['AMD Ryzen 5 4600U','AMD Ryzen 5 3500C'],'AMD')

In [ ]:
df = pd.get_dummies(df, columns=['Resolution','Brand','Color','Processor_Brand'],drop_first=True)

In [ ]:
def linear_regression_score(X, y, kfold:int=5, scoring_type:str='r2'):
    lm = LinearRegression()
    result = cross_val_score(lm, X, y, cv=kfold, scoring=scoring_type)
    return np.mean(result)

def lasso_score(X, y, X_val,y_val ,kfold:int=5, scoring_type:str='r2'):
    alphas = 10**np.linspace(6,-2,50)*0.5
    lasso = Lasso(max_iter=10000, normalize=True)
    
    lassocv = LassoCV(alphas=None, cv=kfold, max_iter=100000, normalize=True)
    lassocv.fit(X_val, y_val)
    lasso.set_params(alpha=lassocv.alpha_)
    clf = Lasso(alpha=lassocv.alpha_)
    scores = cross_val_score(clf, X, y, cv=kfold)
    return scores.mean()
    
def riggid_score(X,y,X_val,y_val,kfold:int=5):
    alphas = 10**np.linspace(6,-2,50)*0.5
    ridgecv = RidgeCV(alphas=alphas, normalize=True)
    ridgecv.fit(X_val, y_val)
    ridge6 = Ridge(alpha=ridgecv.alpha_, normalize=True)
    ridge6.fit(X, y)
    score = cross_val_score(ridge6, X, y, cv=kfold)
    return score.mean()
    
def poly_score(X,y,X_val,y_val,degree:int=2,kfold:int=5):
    poly_features = PolynomialFeatures(degree=degree)
    X_poly = poly_features.fit_transform(X_val)
    poly = LinearRegression()
    return np.mean(cross_val_score(poly, X_poly, y_val, cv=kfold))

In [ ]:
def scores(df, target:str):
    X, X_test, y, y_test = train_test_split(df.loc[:, df.columns != target], df[target], test_size=0.25, random_state=6)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=6)
    linear= linear_regression_score(X_train, y_train)
    lasso = lasso_score(X_train, y_train,X_val,y_val,kfold=10)
    riggid = riggid_score(X_train, y_train,X_val,y_val,kfold=10)
    poly = poly_score(X_train, y_train,X_val,y_val,degree=2,kfold=10)
    return {'Linear Score':linear,'Lasso Score':lasso,'Ridge Score':riggid,'Polynomial Score':poly}

In [ ]:
scores(df, 'Price')